In [1]:
def str_to_digits(s, base=10):
   
    return [int(ch) for ch in s[::-1]]

def digits_to_str(digits, base=10):
    
    if not digits:
        return "0"
    
    while len(digits) > 1 and digits[-1] == 0:
        digits.pop()
    return ''.join(str(d) for d in digits[::-1])

In [2]:
def add(u_str, v_str, base=10):
    u = str_to_digits(u_critical := u_str or "0", base)
    v = str_to_digits(v_critical := v_str or "0", base)
    n = max(len(u), len(v))
    w = []
    carry = 0
    for i in range(n):
        digit_u = u[i] if i < len(u) else 0
        digit_v = v[i] if i < len(v) else 0
        total = digit_u + digit_v + carry
        w.append(total % base)
        carry = total // base
    if carry:
        w.append(carry)
    return digits_to_str(w, base)

In [3]:
def subtract(u_str, v_str, base=10):
    if u_str == v_str:
        return "0"
    u = str_to_digits(u_str, base)
    v = str_to_digits(v_str, base)
    
    w = []
    borrow = 0
    for i in range(len(u)):
        digit_u = u[i] - borrow
        digit_v = v[i] if i < len(v) else 0
        borrow = 0
        if digit_u < digit_v:
            digit_u += base
            borrow = 1
        w.append(digit_u - digit_v)
    return digits_to_str(w, base)

In [4]:
def multiply_schoolbook(u_str, v_str, base=10):
    if u_str == "0" or v_str == "0":
        return "0"
    u = str_to_digits(u_str, base)
    v = str_to_digits(v_str, base)
   
    w = [0] * (len(u) + len(v))
    
    for j in range(len(v)):
        if v[j] == 0:
            continue
        carry = 0
        for i in range(len(u)):
            temp = u[i] * v[j] + w[i + j] + carry
            w[i + j] = temp % base
            carry = temp // base
        if carry:
            w[len(u) + j] += carry
    return digits_to_str(w, base)

In [5]:
def multiply_fast(u_str, v_str, base=10):
    if u_str == "0" or v_str == "0":
        return "0"
    u = str_to_digits(u_str, base)
    v = str_to_digits(v_str, base)
    m, n = len(u), len(v)
    w = [0] * (m + n)
    total_carries = 0

    
    for s in range(m + n - 1):
        total = 0
        
        for i in range(max(0, s - n + 1), min(s + 1, m)):
            j = s - i
            if j < n:
                total += u[i] * v[j]
        total += w[s]  
        w[s] = total % base
        carry = total // base
        if carry:
            if s + 1 >= len(w):
                w.append(0)
            w[s + 1] += carry

   
    i = 0
    while i < len(w):
        if w[i] >= base:
            carry = w[i] // base
            w[i] %= base
            if i + 1 >= len(w):
                w.append(0)
            w[i + 1] += carry
        i += 1

    return digits_to_str(w, base)

In [6]:
def compare(u_digits, v_digits):
    
    while len(u_digits) > 1 and u_digits[-1] == 0:
        u_digits.pop()
    while len(v_digits) > 1 and v_digits[-1] == 0:
        v_digits.pop()
    if len(u_digits) != len(v_digits):
        return len(u_digits) - len(v_digits)
    for i in range(len(u_digits)-1, -1, -1):
        if u_digits[i] != v_digits[i]:
            return u_digits[i] - v_digits[i]
    return 0

def divide(u_str, v_str, base=10):
    if v_str == "0":
        raise ValueError("Division by zero")
    if u_str == "0":
        return "0", "0"
    
    u = str_to_digits(u_str, base)[::-1]  
    v = str_to_digits(v_str, base)[::-1]
    
    
    while len(u) > 1 and u[0] == 0:
        u.pop(0)
    while len(v) > 1 and v[0] == 0:
        v.pop(0)

    n = len(u)
    t = len(v)
    q = [0] * (n - t + 1)
    r = u[:]  
    
    v_shifted = v + [0] * (n - t)

    
    def ge(a, b):
        # a, b 
        a = a[:]
        b = b[:]
        while len(a) > 1 and a[0] == 0: a.pop(0)
        while len(b) > 1 and b[0] == 0: b.pop(0)
        if len(a) != len(b):
            return len(a) > len(b)
        return a >= b

    
    def sub_highfirst(a, b):
        # a >= b
        a = a[:]
        b = b[:]
        
        b = [0]*(len(a)-len(b)) + b
        borrow = 0
        res = [0]*len(a)
        for i in range(len(a)-1, -1, -1):
            val = a[i] - borrow - b[i]
            if val < 0:
                val += base
                borrow = 1
            else:
                borrow = 0
            res[i] = val
        return res

    
    for i in range(n - t + 1):
        
        q_i = 0
        v_current = v + [0] * (n - t - i)  # v * base^(n-t-i)
        while ge(r, v_current):
            r = sub_highfirst(r, v_current)
            q_i += 1
        q[i] = q_i
        
        while len(r) > 1 and r[0] == 0:
            r.pop(0)

    
    while len(q) > 1 and q[0] == 0:
        q.pop(0)
    while len(r) > 1 and r[0] == 0:
        r.pop(0)

    q_str = ''.join(str(x) for x in q)
    r_str = ''.join(str(x) for x in r)
    return q_str, r_str

In [7]:
if __name__ == "__main__":
   
    print("123 + 456 =", add("123", "456"))  

   
    print("456 - 123 =", subtract("456", "123")) 

  
    print("123 * 456 =", multiply_schoolbook("123", "456")) 

    
    q, r = divide("56088", "123")
    print("56088 ÷ 123 = ", q, "Остаток", r)  

123 + 456 = 579
456 - 123 = 333
123 * 456 = 56088
56088 ÷ 123 =  456 Остаток 0
